In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [90]:
df = pd.read_csv('Cancer_Data.csv')

# First look at our data (no vizualization cus im too lazy)

In [91]:
df.sample(20)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
297,892189,M,11.760,18.14,75.00,431.1,0.09968,0.05914,0.02685,0.035150,...,23.39,85.10,553.6,0.11370,0.07974,0.06120,0.07160,0.1978,0.06915,NaN
151,871001502,B,8.219,20.70,53.27,203.9,0.09405,0.13050,0.13210,0.021680,...,29.72,58.08,249.8,0.16300,0.43100,0.53810,0.07879,0.3322,0.14860,NaN
551,923780,B,11.130,22.44,71.49,378.4,0.09566,0.08194,0.04824,0.022570,...,28.26,77.80,436.6,0.10870,0.17820,0.15640,0.06413,0.3169,0.08032,NaN
368,9011971,M,21.710,17.25,140.90,1546.0,0.09384,0.08562,0.11680,0.084650,...,26.44,199.50,3143.0,0.13630,0.16280,0.28610,0.18200,0.2510,0.06494,NaN
268,8910506,B,12.870,16.21,82.38,512.2,0.09425,0.06219,0.03900,0.016150,...,23.64,89.27,597.5,0.12560,0.18080,0.19920,0.05780,0.3604,0.07062,NaN
101,862722,B,6.981,13.43,43.79,143.5,0.11700,0.07568,0.00000,0.000000,...,19.54,50.41,185.2,0.15840,0.12020,0.00000,0.00000,0.2932,0.09382,NaN
474,911391,B,10.880,15.62,70.41,358.9,0.10070,0.10690,0.05115,0.015710,...,19.35,80.78,433.1,0.13320,0.38980,0.33650,0.07966,0.2581,0.10800,NaN
417,90602302,M,15.500,21.08,102.90,803.1,0.11200,0.15710,0.15220,0.084810,...,27.65,157.10,1748.0,0.15170,0.40020,0.42110,0.21340,0.3003,0.10480,NaN
267,8910499,B,13.590,21.84,87.16,561.0,0.07956,0.08259,0.04072,0.021420,...,30.04,97.66,661.5,0.10050,0.17300,0.14530,0.06189,0.2446,0.07024,NaN
217,8811779,B,10.200,17.48,65.05,321.2,0.08054,0.05907,0.05774,0.010710,...,24.47,75.40,403.7,0.09527,0.13970,0.19250,0.03571,0.2868,0.07809,NaN


# Let's see if there are null values in our data

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

# Let's clean data by simply dropping useless columns

In [93]:
df = df.drop(['id','Unnamed: 32'], axis=1)

# Feed all data to our model ignoring high correlation between some of our features

In [94]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Split our data to features (X) and target (Y)

In [95]:
X = df.drop('diagnosis',axis=1)
Y = df['diagnosis']

# Defining transformations for our categorical and numerical data

In [96]:
num_transformer = Pipeline(
    steps=[('scale',StandardScaler())]
)
cat_transformer = Pipeline(
    steps=[('code',OneHotEncoder())]
)
processor = ColumnTransformer(
    transformers=[
        ('num_transf', num_transformer, selector(dtype_exclude="category")),
        ('cat_transf', cat_transformer, selector(dtype_include="category"))
    ]
)

# Creating model in a pipeline class

In [97]:
clf = Pipeline(steps=[
    ('transform_data', processor),
    ('model', RandomForestClassifier())
])

# Splitting data into training and test sets

In [98]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# Fitting and evaluating our model

In [99]:
clf.fit(X_train, Y_train)
print("model score: %.3f" % clf.score(X_test, Y_test))

model score: 0.947
